In [ ]:
import sys
sys.path.append('../python_packages_static/')
import pandas as pd
import pyemu
import os
import numpy as np

## Setting parameter values from results of prior_mc_wide rejection sampling (notebook 1.4)

### NOTE: Make sure  `run_ensemble` is set appropriately - If `run_ensemble` is set to `True`, local runs are performed. If `run_ensemble` set to `False`results from the journal article are used. 

In [ ]:
run_ensemble=False

In [ ]:
if run_ensemble==True:
    noptmax0_dir = '../noptmax0_testing' # directory to write out the noptmax=0 iES file to for running to reqeight
else:
    noptmax0_dir = '../output/noptmax0'
wkdir = '../run_data/' # set the PEST++ run directory    
pstroot='prior_mc_wide' 


In [ ]:
pst = pyemu.Pst(os.path.join(noptmax0_dir,'{}.pst'.format(pstroot)))

## read in the parameter ensemble that made it through rejection sampling

In [ ]:
parens = pd.read_csv(os.path.join(wkdir, 'never_iES_post_MC_parens.csv'), index_col=0)

## use the `summary` function in `pandas` for the parameter ensemble to find the quantiles we use for lower and upper bounds and the starting value

In [ ]:
summary= parens.describe()

In [ ]:
assert np.unique(summary.T.index == pst.parameter_data.index)

In [ ]:
pst.parameter_data.parlbnd = summary.T['25%']
pst.parameter_data.parval1 = summary.T['50%']
pst.parameter_data.parubnd = summary.T['75%']

In [ ]:
pst.control_data.noptmax = 0

In [ ]:
pst.write(os.path.join(noptmax0_dir, 'never_iES_noptmax0.pst'))

## now run local noptmax=0

In [ ]:
if run_ensemble==True:
    cwd = os.getcwd()
    os.chdir('../noptmax0_testing/')
    pyemu.utils.os_utils.run(f'pestpp-ies never_iES_noptmax0.pst')
    print(os.getcwd())
    os.chdir(cwd)